In [ ]:
import torch
import numpy as np
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
#import the data
from htru1 import HTRU1

#Transforms the data. This seems like a required step as I kepts getting errors if I did not include it
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

#Specifically import the training data and test data
train_data = HTRU1('data', train=True, download=True, transform=transform)
test_data = HTRU1('data', train=False, download=True, transform=transform)

#Variable setup to extract the data
trainloader = torch.utils.data.DataLoader(train_data, batch_size=50000, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=10000, shuffle=True, num_workers=2)

#Extracts the data into the features and the labels
dataiter = iter(trainloader)
train_tensor_images, train_tensor_labels = dataiter.next()
dataiter = iter(testloader)
test_tensor_images, test_tensor_labels = dataiter.next()

#Converts the data to np arrays (found them easier to work with)
train_features = np.array(train_tensor_images)
train_label = np.array(train_tensor_labels)
test_features = np.array(test_tensor_images)
test_label = np.array(test_tensor_labels)

In [ ]:
#transformed the feates from (5000, 3, 32, 32) into (3, 5000, 32, 32)
train_features_t = np.transpose(train_features, (1,0,2,3))
test_features_t = np.transpose(test_features, (1,0,2,3))

In [ ]:
#Assigning each layer into their own variable
train_layer1 = train_features_t[0]
train_layer2 = train_features_t[1]
train_layer3 = train_features_t[2]
test_layer1 = test_features_t[0]
test_layer2 = test_features_t[1]
test_layer3 = test_features_t[2]

In [ ]:
#Flattening the data
ndims = train_layer1.shape[1] * train_layer1.shape[2]
X_train = train_layer1.reshape(train_layer1.shape[0],ndims)
X_test = test_layer1.reshape(test_layer1.shape[0],ndims)

In [ ]:
X_train_2 = train_layer1.reshape(train_layer2.shape[0],ndims)
X_test_2 = test_layer1.reshape(test_layer2.shape[0],ndims)

X_train_3 = train_layer1.reshape(train_layer3.shape[0],ndims)
X_test_3 = test_layer1.reshape(test_layer3.shape[0],ndims)

In [ ]:
#renaming the y data
y_train = train_label
y_test  = test_label

In [ ]:
#Importing tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
def build_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
def compile_model(model,metric):
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[metric])
    return model

In [ ]:
def fit_model(model,epochs):
    model.fit(
    X_train_3,
    y_train,
    epochs=epochs,
    shuffle=True,
    verbose=1
    )
    return model

In [ ]:
def run_model(metric,epoch):
    built_model = build_model()
    compiled_model = compile_model(built_model,metric)
    model = fit_model(compiled_model,epoch)
    y_pred = model.predict(X_test).round()
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    TPR = cm[0][0]/(cm[0][0]+cm[1][0])
    print((TPR.round(2))*100)
    accuracy_score(y_test, y_pred)
    print(accuracy_score)
    #return model

In [ ]:
#y_pred = run_model('accuracy',10)
run_model('accuracy',10)

In [ ]:
run_model('accuracy',25)

In [ ]:
run_model('Recall',15)

In [ ]:
run_model('TruePositives',15)

In [ ]:
run_model('Precision',15)